In [270]:
import json
import numpy as np
import pickle
# import pandas as pd
# from tqdm  import tqdm

import json, bz2, os
import sys

from tqdm import tqdm
# from ase_write import write_xyz
import pandas as pd
import numpy as np
from pymatgen.core.structure import Structure
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')
from m3gnet.models import M3GNet, Potential
# from sklearn.model_selection import train_test_split
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.structure import Structure

In [18]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 100})

In [19]:
e_cutoff = 1e10

In [20]:
m3gnet = M3GNet.load()
potential = Potential(model=m3gnet)

def get_data(data, keys, e_cutoff=0., r_cutoff=5.0):
    structures = []
    energy = []
    stress = []
    forces = []
    counter = 0
    isolated_systems = []
    for el, key in tqdm(zip(data, keys)):
        if len(el)==0:
            continue
        # select all structures with an energy difference of more than e_cutoff to their previous structure
        N = len(el[0]['structure']['sites'])
        e_0 = el[0]['energy']/N
        indices = []
        for idx, sys in enumerate(el):
            if abs(sys['energy']/N - e_0) > e_cutoff or idx==0 or idx==len(el)-1:
                e_0 = sys['energy']/N
                indices.append(idx)
        if len(indices)>2:
            pass
        elif len(el)==3:
            indices  = range(3)
        elif len(el)>3:
            indices = [0,len(el)//2 ,len(el)-1]
        else:
            indices = range(len(el))
        for index in indices:
            if len(el[index]['structure']['sites'])<20 and np.max(np.abs(el[index]['stress']))<500 and np.max(np.abs(el[index]['forces']))<20. and el[index]['energy']<0 and el[index]['energy'] > -20.:
                try:
                    st = Structure.from_dict(el[index]['structure'])
                    if not any([len(neigh) for neigh in st.get_all_neighbors(r=5.)]):
                        float('sdfas')
                    potential.graph_converter.convert(st)                                        
                    structures.append(st)
                    energy.append(el[index]['energy'])
                    stress.append(-0.1*np.array(el[index]['stress']))  #kbar to GPa
                    forces.append(el[index]['forces'])
                except:
                    isolated_systems.append([key, index])
                    print(f'Isolated system {key} found')
    return energy,forces,stress,structures, isolated_systems

In [443]:
def create_relaxation_dataset(run_keys):
    run_files = []
    
    ## Implement method that also exports the run keys that were succesful 
    
    print(len(run_keys))
    
    for file in tqdm(sorted(os.listdir("data_prep/geo_opt"))):
        if file.endswith('json.bz2'):
  
            batch = json.load(bz2.open("data_prep/geo_opt/{}".format(file), 'r'))
            print("Length batch: {}".format(len(batch)))
        
            batch_keys = list(batch.keys())
            
            key_intersection = list(set(batch_keys).intersection(run_keys))
            print(key_intersection)
            
            for key in key_intersection:
                try:
                    reformated_run = {key:[batch[key][0], batch[key][-1]]} # for each run only select the first and last geometry optimization
                    # print(reformated_run)
                    run_files.append(reformated_run)
                except:
                    if len(batch[key])<1:
                        # print(batch[key])
                        print("Array too small")
                    print("Error")

    return run_files

### Get the relaxed Structures (test structures)

In [21]:
# pickle_file_path = 'relaxation_10000000000/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'

relaxed_structures_path = 'relaxations/0_f_t-2023-05-22_10-41-22/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'

# Load the pickle file
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [51]:
# print(relaxed_structures.keys())
runs_relaxed = list(relaxed_structures.keys()) #this is list of differnet runs
print(runs_relaxed[:10])
selected_run = runs_relaxed[0]

['runs_2/spg70/O/O5Ta4/xxx_02p-00_70-2-O3c.O2b.Ta3c.Ta1a', 'runs_ml_3/ABC4/runs/batch-000/Mn/MnBr4Dy/xxx_02a-00_agm200032544_spg11', 'runs_2/spg31/P/PAg/xxx_02a-00_31-2-P4c.Ag4c', 'runs_3/spg13/F/F3K2Ag/xxx_02a-00_13-3-F4b.F2a.K4b.Ag2a', 'runs_2/spg73/As/AsIn3/xxx_02a-00_73-2-As2b.As1a.In6d.In3c', 'runs_2/spg21/N/NPd/xxx_02a-00_21-2-N4c.Pd4c', 'runs_ml_2/AB2/runs/batch-000/K/K2Tl/xxx_02a-00_agm200014232_spg53', 'runs_2/spg73/B/BI3/xxx_02a-00_73-2-B2b.B1a.I6d.I2b.I1a', 'runs_3/spg13/Br/Br3Sr2I/xxx_02a-00_13-3-Br4b.Br2a.Sr4b.I2a', 'runs_2/spg70/F/F4Mn/xxx_02a-00_70-2-F6d.F3c.F2b.F1a.Mn2b.Mn1a']


In [ ]:
# # Old Method
# run_files = []
# wd = os.getcwd()
# # os.chdir('geo_opt')

# for file in tqdm(sorted(os.listdir("data_prep/geo_opt"))[0:2]):
#     if file.endswith('json.bz2'):
#         run_files.append(json.load(bz2.open("data_prep/geo_opt/{}".format(file), 'r')))

In [49]:
# run_files = []
# wd = os.getcwd()
# # os.chdir('geo_opt')

# for file in tqdm(sorted(os.listdir("data_prep/geo_opt"))[0:2]):
#     if file.endswith('json.bz2'):
#         # for each run only select the first and last geometry optimization
#         batch = json.load(bz2.open("data_prep/geo_opt/{}".format(file), 'r'))
#         print(len(batch))
#         keys_batch = list(batch.keys())
#         print(batch["runs_1/elements/Ag/Ag/xxx_02k-00_phexAl"][0])
#         for key in keys_batch:
#             print(key)
#             reformated_run = {key: [batch[key][0], batch[key][-1]]}
#             print(reformated_run)

  0%|                                                                                                                                                                                                                                                                    | 0/2 [00:03<?, ?it/s]

532


IndexError: list index out of range

In [137]:
reformated_run = create_relaxation_dataset(runs_relaxed)

4077


  1%|██▋                                                                                                                                                                                                                                                        | 1/92 [00:01<02:36,  1.72s/it]

Length batch: 532
['runs_1/elements/Mn/Mn/xxx_02k-00_psqr2Be', 'runs_1/elements/Cl/Cl/xxx_02k-00_psqrAs', 'runs_1/elements/In/In/xxx_02p-00_graphene', 'runs_1/elements/Y/Y/xxx_02p-00_graphene', 'runs_1/elements/Re/Re/xxx_02k-00_psqr2Be', 'runs_1/elements/N/N/xxx_02k-00_skepler', 'runs_1/elements/B/B/xxx_02k-00_phexAl', 'runs_1/elements/Ce/Ce/xxx_02p-00_graphene', 'runs_1/elements/S/S/xxx_02k-00_skepler', 'runs_1/elements/Er/Er/xxx_02p-00_graphene', 'runs_1/elements/Co/Co/xxx_02p-00_graphene', 'runs_1/elements/Be/Be/xxx_02k-00_phexAl', 'runs_1/elements/Fe/Fe/xxx_02k-00_skepler', 'runs_1/elements/Ta/Ta/xxx_02k-00_skepler', 'runs_1/elements/Fe/Fe/xxx_02k-00_phexAl', 'runs_1/elements/Cl/Cl/xxx_02k-00_pSsBr', 'runs_1/elements/Re/Re/xxx_02k-00_skepler', 'runs_1/elements/In/In/xxx_02k-00_phexAl', 'runs_1/elements/S/S/xxx_02p-00_graphene', 'runs_1/elements/Ho/Ho/xxx_02k-00_psqrAs']
Array too small
Error
Array too small
Error
Array too small
Error


  2%|█████▍                                                                                                                                                                                                                                                     | 2/92 [00:02<01:31,  1.01s/it]

Length batch: 22
[]


...

Length batch: 42
['runs_ml_4/AB2C2D2/runs/batch-000/Co/Co2Se2Br2Pa/xxx_02a-00_agm200015997_spg164', 'runs_ml_4/AB2C2D2/runs/batch-000/Cl/Cl2Se2In2Sn/xxx_02a-00_agm200016086_spg164', 'runs_ml_4/AB2C2D2/runs/batch-000/Cl/Cl2Ni2Se2Pa/xxx_02a-00_agm200016110_spg164', 'runs_ml_4/AB2C2D2/runs/batch-000/Ti/Ti2VSe2Br2/xxx_02a-00_agm200015908_spg164']


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 91/92 [19:46<00:12, 12.85s/it]

Length batch: 469
[]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [19:52<00:00, 12.97s/it]

Length batch: 923
['runs_ml_4/ABC2D6/runs/batch-000/Na/NaMgCu2I6/xxx_02a-00_agm200031312_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Li/LiO6P2Cu/xxx_02a-00_agm200030811_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/O/O6CuSe2Rb/xxx_02a-00_agm200030770_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Na/NaSi2Se6Cd/xxx_02a-00_agm200030957_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Cu/Cu2Br6AgNp/xxx_02a-00_agm200030907_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Na/NaP2GaSe6/xxx_02a-00_agm200031123_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Na/NaS6As2Ce/xxx_02a-00_agm200031453_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Li/LiCu2Br6Ag/xxx_02a-00_agm200030868_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Ni/NiCu2Br6Ag/xxx_02a-00_agm200030865_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Na/NaAlP2S6/xxx_02a-00_agm200031476_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/Na/NaCl6Ag2Cd/xxx_02a-00_agm200031024_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/P/P2S6MnAg/xxx_02a-00_agm200031402_spg149', 'runs_ml_4/ABC2D6/runs/batch-000/

Save the reformated testing data set

In [282]:
pickle.dump(reformated_run, open("reformated_struc_dict.pickle", 'wb'))

In [439]:
print(len(reformated_run))

4074


### Get the relaxation structures (validation)

In [441]:
relaxed_structures_path_val = 'relaxations/save_model_validation/save_model/no_cutoff_lr_1e-3_bs_128/00496-0.118736-0.038684-0.073790-0.062614relax_struc_0.1_1000_val.pickle'

# Load the pickle file
with open(relaxed_structures_path_val, 'rb') as file:
    relaxed_structures_val = pickle.load(file)

In [442]:
runs_relaxed_val = list(relaxed_structures_val.keys()) #this is list of differnet runs
print(runs_relaxed_val[:10])
selected_run = runs_relaxed_val[0]

['runs_2/spg43/S/S3Sb2/xxx_02a-00_43-2-S8d.S4c.Sb8d', 'runs_ml_4/ABC2D6/runs/batch-000/P/P2S6TiAg/xxx_02a-00_agm200030743_spg149', 'runs_ext/c2db_0.25_1/C/C/xxx_02a-00_C2-a6735a4a3797', 'runs_ml_2/A2B3/runs/batch-000/Co/Co2Sm3/xxx_02a-00_agm200003071_spg18', 'runs_ml_3/ABC/runs/batch-000/Br/BrTeBi/xxx_02a-00_agm200029360_spg59', 'runs_2/spg30/Cl/Cl3Gd/xxx_02p-00_30-2-Cl4c.Cl2a.Gd2a', 'runs_3/spg13/Be/Be2P3Re/xxx_02a-00_13-3-Be4b.P4b.P2a.Re2a', 'runs_2/spg13/S/SPd/xxx_02a-00_13-2-S4b.Pd4b', 'runs_ml_3/ABC4/runs/batch-000/As/AsNbTe4/xxx_02a-00_agm200033718_spg11', 'runs_2/spg70/Cd/CdTe/xxx_02p-00_70-2-Cd3c.Te2b.Te1a']


In [ ]:
reformated_run_val = create_relaxation_dataset(runs_relaxed_val)

4174


  1%|██▋                                                                                                                                                                                                                                                        | 1/92 [00:07<11:13,  7.40s/it]

Length batch: 532
['runs_1/elements/Tl/Tl/xxx_02k-00_phexAl', 'runs_1/elements/Al/Al/xxx_02k-00_psqr2Be', 'runs_1/elements/Rh/Rh/xxx_02p-00_graphene', 'runs_1/elements/Hg/Hg/xxx_02k-00_phexAl', 'runs_1/elements/V/V/xxx_02k-00_phexAl', 'runs_1/elements/Ca/Ca/xxx_02k-00_dkepler', 'runs_1/elements/Au/Au/xxx_02k-00_dkepler', 'runs_1/elements/Au/Au/xxx_02p-00_graphene', 'runs_1/elements/I/I/xxx_02k-00_phexAl', 'runs_1/elements/Ge/Ge/xxx_02k-00_phexAl', 'runs_1/elements/Pr/Pr/xxx_02k-00_psqrAs', 'runs_1/elements/W/W/xxx_02k-00_psqrAs', 'runs_1/elements/La/La/xxx_02k-00_skepler', 'runs_1/elements/Mg/Mg/xxx_02k-00_dhexII', 'runs_1/elements/In/In/xxx_02k-00_psqrAs', 'runs_1/elements/Pd/Pd/xxx_02k-00_dkepler', 'runs_1/elements/Br/Br/xxx_02k-00_phexAl', 'runs_1/elements/Rb/Rb/xxx_02p-00_graphene', 'runs_1/elements/Sm/Sm/xxx_02k-00_psqrAs', 'runs_1/elements/V/V/xxx_02k-00_skepler', 'runs_1/elements/Os/Os/xxx_02k-00_psqrAs', 'runs_1/elements/Cl/Cl/xxx_02p-00_spg63_4', 'runs_1/elements/Se/Se/xxx_02p

  2%|█████▍                                                                                                                                                                                                                                                     | 2/92 [00:08<05:12,  3.48s/it]

Length batch: 22
['runs_2/carbon/C/CCl3/xxx_02a-00_4-2-C1a.Cl2b.Cl1a']


...

Length batch: 3299
['runs_2/spg15/S/SHg/xxx_02a-00_15-2-S2b.Hg2c', 'runs_2/spg15/Cl/ClAg/xxx_02a-00_15-2-Cl4d.Ag2c.Ag2b', 'runs_2/spg15/F/F2Au/xxx_02a-00_15-2-F4d.F2c.F2b.Au2c.Au2a', 'runs_2/spg15/Cu/Cu3Se2/xxx_02a-00_15-2-Cu2c.Cu2b.Cu2a.Se4d', 'runs_2/spg15/F/F2Pd/xxx_02a-00_15-2-F2c.F2a.Pd2c', 'runs_2/spg15/F/F2Nb/xxx_02a-00_15-2-F4d.F2b.F2a.Nb4d', 'runs_2/spg15/Fe/Fe2Te3/xxx_02a-00_15-2-Fe2c.Fe2b.Te4d.Te2a', 'runs_2/spg15/I/I3La/xxx_02a-00_15-2-I4d.I2c.La2b', 'runs_2/spg15/Ag/Ag2Te/xxx_02a-00_15-2-Ag2c.Ag2a.Te2c', 'runs_2/spg15/Li/LiI/xxx_02a-00_15-2-Li4d.Li2a.I4d.I2c', 'runs_2/spg15/S/S2Hf/xxx_02a-00_15-2-S4d.Hf2a', 'runs_2/spg15/Fe/FeI2/xxx_02a-00_15-2-Fe2b.I4d', 'runs_2/spg15/Se/Se3Nd2/xxx_02a-00_15-2-Se4d.Se2c.Nd2c.Nd2a', 'runs_2/spg15/F/FIn/xxx_02a-00_15-2-F2c.F2a.In2c.In2b', 'runs_2/spg15/N/NFe/xxx_02a-00_15-2-N2c.N2a.Fe2c.Fe2b', 'runs_2/spg15/N/NEu/xxx_02a-00_15-2-N4d.Eu4d', 'runs_2/spg15/Br/Br3Yb/xxx_02a-00_15-2-Br4d.Br2c.Yb2a', 'runs_2/spg15/F/F2Cd/xxx_02a-00_15-2-F2c.F2b.C

 16%|████████████████████████████████████████▊                                                                                                                                                                                                                 | 15/92 [02:16<16:47, 13.08s/it]

Length batch: 1305
['runs_2/spg17/Br/Br3Tl/xxx_02a-00_17-2-Br4c.Br2a.Tl2b', 'runs_2/spg17/O/OCu2/xxx_02a-00_17-2-O2b.Cu4c', 'runs_2/spg17/Br/Br3Sm/xxx_02a-00_17-2-Br4c.Br2a.Sm2b', 'runs_2/spg17/N/NTa/xxx_02a-00_17-2-N4c.N2b.Ta4c.Ta2a', 'runs_2/spg17/Cl/Cl2Fe/xxx_02a-00_17-2-Cl4c.Fe2b', 'runs_2/spg17/Li/LiBr/xxx_02a-00_17-2-Li4c.Br2b.Br2a', 'runs_2/spg17/Cl/Cl2Hg/xxx_02a-00_17-2-Cl4c.Cl2b.Cl2a.Hg4c', 'runs_2/spg17/Fe/FeTe2/xxx_02a-00_17-2-Fe2b.Te4c', 'runs_2/spg17/N/NAg/xxx_02a-00_17-2-N4c.Ag2b.Ag2a', 'runs_2/spg17/S/SCa/xxx_02a-00_17-2-S4c.Ca2b.Ca2a', 'runs_2/spg17/Cu/CuTe/xxx_02a-00_17-2-Cu2b.Cu2a.Te4c', 'runs_2/spg17/P/PTb/xxx_02a-00_17-2-P4c.Tb2b.Tb2a', 'runs_2/spg17/O/OMn/xxx_02a-00_17-2-O2a.Mn2b', 'runs_2/spg17/P/PV/xxx_02a-00_17-2-P2b.P2a.V4c', 'runs_2/spg17/Se/Se3Ir/xxx_02a-00_17-2-Se4c.Se2a.Ir2b', 'runs_2/spg17/O/OAg2/xxx_02a-00_17-2-O4c.Ag4c.Ag2b.Ag2a', 'runs_2/spg17/P/PAg3/xxx_02a-00_17-2-P2b.Ag4c.Ag2a', 'runs_2/spg17/F/F3Eu/xxx_02a-00_17-2-F4c.F2b.Eu2a', 'runs_2/spg17/P/PEu/

 17%|███████████████████████████████████████████▍                                                                                                                                                                                                              | 16/92 [02:31<17:01, 13.44s/it]

Length batch: 1108
['runs_2/spg21/Be/BeCl2/xxx_02a-00_21-2-Be2a.Cl4c', 'runs_2/spg21/Br/Br2Hg/xxx_02a-00_21-2-Br2b.Br2a.Hg2a', 'runs_2/spg21/P/P2Mn3/xxx_02a-00_21-2-P4c.Mn4c.Mn2b', 'runs_2/spg21/Cr/Cr3Se4/xxx_02a-00_21-2-Cr4c.Cr2b.Se4c.Se2b.Se2a', 'runs_2/spg21/Co/CoAs/xxx_02a-00_21-2-Co4c.Co2b.Co2a.As4c.As2b.As2a', 'runs_2/spg21/In/InTe/xxx_02a-00_21-2-In2b.Te2b', 'runs_2/spg21/Cl/Cl3Ni/xxx_02a-00_21-2-Cl4c.Cl2b.Ni2b', 'runs_2/spg21/N/NAu/xxx_02a-00_21-2-N4c.N2b.Au4c.Au2a', 'runs_2/spg21/S/S3Ga2/xxx_02a-00_21-2-S4c.S2b.Ga2b.Ga2a', 'runs_2/spg21/Li/Li2O/xxx_02a-00_21-2-Li4c.O2b', 'runs_2/spg21/Cu/CuTe/xxx_02a-00_21-2-Cu4c.Te4c', 'runs_2/spg21/Zn/Zn3As2/xxx_02a-00_21-2-Zn4c.Zn2b.As4c', 'runs_2/spg21/Rb/RbTl/xxx_02a-00_21-2-Rb2a.Tl2a', 'runs_2/spg21/Sc/ScBr3/xxx_02a-00_21-2-Sc2a.Br4c.Br2a', 'runs_2/spg21/Br/Br2Pb/xxx_02a-00_21-2-Br4c.Pb2b', 'runs_2/spg21/F/F3Sc/xxx_02a-00_21-2-F4c.F2b.Sc2a', 'runs_2/spg21/P/P2Re/xxx_02a-00_21-2-P2b.P2a.Re2a', 'runs_2/spg21/Ca/CaBr2/xxx_02a-00_21-2-Ca2b.B

## Relaxation Error

In [308]:
def mae(arr):
    n = len(arr)
    # print(n)
    return sum(arr)/n

def rmse(arr):
    n = len(arr)
    return np.sqrt(sum(np.square(arr))/n)


def area_per_atom(lat_param, nr_atoms):
    area = lat_param[0]*lat_param[1] #yields the Area in Angstrom^2
    res_area_per_atom = area/nr_atoms
    
    return res_area_per_atom

def area_per_atom_sqrt(lat_param, nr_atoms):
    area = lat_param[0]*lat_param[1] #yields the Area in Angstrom^2
    res_area_per_atom = np.sqrt(area)/nr_atoms
    
    return res_area_per_atom

def get_error(relaxed_structures, train_structures):
    
    runs_compared = 0
    runs_compared_volume = 0
    
    runs_relaxed = list(relaxed_structures.keys()) #this is list of differnet runs
    
    diff_energy_per_atom = []
    diff_area_per_atom = []
    diff_area_per_atom_sqrt = []
    area_per_atom_sqrt_arr = []
    
    for key in runs_relaxed:
        # print("key: {}".format(key))
        for dictionary in train_structures:
            if key in dictionary:
                # CHECK IF Energy per atom or Energy 
                train_run = dictionary
                final_structure_training = train_run[key][-1]
                # print(final_structure_training)
                
                ### Energy
               
                nr_sites = len(Structure.from_dict(final_structure_training["structure"])) #These should be the same to the # of sites in the relaxed structure training set
                final_structure_training_energy = final_structure_training['energy']/nr_sites
                final_structure_relaxation = relaxed_structures[key]
                # print(final_structure_relaxation)
                final_structure_relaxation_energy = final_structure_relaxation['energy']
                
                ### Volume
                
                struct_train = Structure.from_dict(final_structure_training["structure"])
                struct_relax = final_structure_relaxation["final_structure"]
                
                if struct_relax != None:
            
                    lat_parm_train = struct_train.lattice.abc
                    lat_parm_relax = struct_relax.lattice.abc

                    area_per_atom_train_sqrt = area_per_atom_sqrt(lat_parm_train, nr_sites) 
                    area_per_atom_relax_sqrt = area_per_atom_sqrt(lat_parm_relax, nr_sites)
                    
                    area_per_atom_train = area_per_atom(lat_parm_train, nr_sites) 
                    area_per_atom_relax = area_per_atom(lat_parm_relax, nr_sites)
                    
                    area_per_atom_sqrt_arr.append(area_per_atom_relax_sqrt)
                    
                    # print(area_per_atom_train)
                    # print(area_per_atom_relax)
                    
                    diff_area_sqrt = np.abs(area_per_atom_train_sqrt - area_per_atom_relax_sqrt)
                    diff_area_org = np.abs(area_per_atom_train - area_per_atom_relax)
                    
                    diff_area_per_atom_sqrt.append(diff_area_sqrt)
                    diff_area_per_atom.append(diff_area_org)
                    
                    runs_compared_volume += 1
            
                
                if final_structure_relaxation_energy != None:
                    final_structure_relaxation_energy = final_structure_relaxation['energy']/nr_sites
                    diff_energy_per_atom.append(np.abs(final_structure_training_energy - final_structure_relaxation_energy))
                    # print(final_structure_training_energy, final_structure_relaxation_energy)
                    runs_compared += 1
                    
    
   
    print("Runs compared: {}".format(runs_compared))
    print("Runs compared volume: {}".format(runs_compared_volume))
    print("Converged Percentage: {}%".format(np.round(runs_compared/len(runs_relaxed),3)*100))
    # print("Length diff_energy: {}".format(len(diff_energy_per_atom)))
    print("Resulting MAE energy: {}".format(mae(diff_energy_per_atom)))
    print("Resulting MAE area per atom: {}".format(mae(diff_area_per_atom)))
    print("Resulting MAE area per atom sqrt: {}".format(mae(diff_area_per_atom_sqrt)))
    print("Mean area per atom sqrt: {}".format(np.mean(area_per_atom_sqrt_arr)))
    

    return

In [309]:
relaxed_structures[list(relaxed_structures.keys())[3]]['energy']

-41.03407669067383

# Data evaluated on test structures

## No Cutoff

### Model 1 lr_1e3_bs_128

In [324]:
# pickle_file_path = 'relaxation_10000000000/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'
relaxed_structures_path = 'relaxations/0_f_t-2023-05-22_10-41-22/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'

# Load the pickle file
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [325]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.9097554642982799
Runs compared: 2983
Runs compared volume: 2983
Converged Percentage: 73.2%
Resulting MAE energy: 0.07131015342398918
Resulting MAE area per atom: 0.7396141282735273
Resulting MAE area per atom sqrt: 0.0501764271998642


#### with $f$_max = 0.1 and $n$-steps = 1000

In [331]:
# pickle_file_path = 'relaxation_10000000000/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'
relaxed_structures_path = 'relaxations/save_model/no_cutoff_lr_1e-3_bs_128/00496-0.118736-0.038684-0.073790-0.062614relaxed_structures_0.1_1000.pickle'

# Load the pickle file
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [332]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.8580160108362819
Runs compared: 4052
Runs compared volume: 4052
Converged Percentage: 99.3%
Resulting MAE energy: 0.08267885711685917
Resulting MAE area per atom: 0.8343885339322874
Resulting MAE area per atom sqrt: 0.05242848025127047


### Model lr_1e3_bs_128_opt

In [326]:
# pickle_file_path = 'relaxation_10000000000/00456-0.120248-0.040146-0.073807-0.062954relaxed_structures.pickle'
# relaxed_structures_path = 'relaxations/save_model/no_cutoff_lr_1e3_bs128/00496-0.118736-0.038684-0.073790-0.062614relaxed_structures.pickle'
relaxed_structures_path = 'relaxations/save_model/no_cutoff_lr_1e-3_bs_128/00496-0.118736-0.038684-0.073790-0.062614relax_structures_0.01_400.pickle'

# Load the pickle file
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [327]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.9067820546836252
Runs compared: 2996
Runs compared volume: 2996
Converged Percentage: 73.4%
Resulting MAE energy: 0.07045344290033809
Resulting MAE area per atom: 0.7094289782814373
Resulting MAE area per atom sqrt: 0.04863491548737956


### Model lr_1e-4_bs_128

In [313]:
relaxed_structures_path = 'relaxations/save_model_daint/save_model/no_cutoff_lr_1e-4_bs_128relaxed_structures_0.1_1000.pickle'

# Load the pickle filerelaxations/save_model_daint/save_model/no_cutoff_lr_1e-4_bs_128relaxed_structures_0.1_1000.pickle
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [314]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.8591135017553148
Runs compared: 4061
Runs compared volume: 4061
Converged Percentage: 98.2%
Resulting MAE energy: 0.0768148361164508
Resulting MAE area per atom: 0.8717012858738827
Resulting MAE area per atom sqrt: 0.05265356030176709


## E_cutoff = 0.01

### Model lr_1e-3_bs_128

In [291]:
relaxed_structures_path = 'relaxations/save_model_daint/save_model/0.01_lr_1e-3_bs_128/00138-0.147841-0.043230-0.096927-0.076845relaxed_structures_0.1_1000.pickle'

# Load the pickle filerelaxations/save_model_daint/save_model/no_cutoff_lr_1e-4_bs_128relaxed_structures_0.1_1000.pickle
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [292]:
get_error(relaxed_structures, train_structures = reformated_run)

Runs compared: 4060
Runs compared volume: 4060
Converged Percentage: 98.2%
Resulting MAE energy: 0.0842301533247498
Resulting MAE area per atom: 0.842659175505702
Resulting MAE area per atom sqrt: 0.051782874010324706


### Model lr_1e-4_bs_128

In [315]:
relaxed_structures_path = "relaxations/save_model_daint/save_model/0.01_lr_1e-4_bs_128/00760-0.147770-0.041342-0.098456-0.079714relaxed_structures_0.1_1000.pickle"

# Load the pickle filerelaxations/save_model_daint/save_model/no_cutoff_lr_1e-4_bs_128relaxed_structures_0.1_1000.pickle
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [316]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.8620878523488482
Runs compared: 4057
Runs compared volume: 4057
Converged Percentage: 98.1%
Resulting MAE energy: 0.0823106985171674
Resulting MAE area per atom: 0.8833721178460983
Resulting MAE area per atom sqrt: 0.05372627605536928


## E_cutoff = 0.02

### Model lr_1e-3_bs_128

In [317]:
relaxed_structures_path = 'relaxations/save_model_daint/save_model/0.02_lr_1e-3_bs_128/00227-0.151426-0.045653-0.097765-0.080075relaxed_structures_0.1_1000.pickle'

with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [318]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.8584192583462572
Runs compared: 4042
Runs compared volume: 4042
Converged Percentage: 97.8%
Resulting MAE energy: 0.08373413153879425
Resulting MAE area per atom: 0.8491564034623496
Resulting MAE area per atom sqrt: 0.053432477992073776


### Model lr_1e-4_bs_128

In [319]:
relaxed_structures_path = "relaxations/save_model_daint/save_model/0.02_lr_1e-4_bs_128/00991-0.149881-0.042665-0.098949-0.082664relaxed_structures_0.1_1000.pickle"

with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [320]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: 0.8610833889096845
Runs compared: 4052
Runs compared volume: 4052
Converged Percentage: 98.0%
Resulting MAE energy: 0.08409416656703546
Resulting MAE area per atom: 0.8657371294493192
Resulting MAE area per atom sqrt: 0.0529658638927524


## MACE Relaxations

### Model lr 3e-3 bs 128

In [431]:
##Model lr 0.003, bs=128

mace_model_path = "relaxation_mace/save_model/MACE_0.003_128_swa.modelrelaxed_structures_0.01_400.pickle"

with open(mace_model_path, 'rb') as file:
    mace_relaxed = pickle.load(file)

In [432]:
get_error(mace_relaxed, train_structures = reformated_run)

Mean area per atom sqrt: 0.842197718719904
Runs compared: 4031
Runs compared volume: 4031
Converged Percentage: 97.7%
Resulting MAE energy: 0.09784329760631653
Resulting MAE area per atom: 0.7451553557813656
Resulting MAE area per atom sqrt: 0.04939235846317204


### Model lr 1e-2 bs 64

In [433]:
mace_model_path = "relaxation_mace/save_model/MACE_0.01_64_swa.modelrelaxed_structures_0.01_400.pickle"

with open(mace_model_path, 'rb') as file:
    mace_relaxed = pickle.load(file)

In [434]:
get_error(mace_relaxed, train_structures = reformated_run)

Mean area per atom sqrt: 0.8420352412621123
Runs compared: 3962
Runs compared volume: 3962
Converged Percentage: 97.2%
Resulting MAE energy: 0.14196816226436285
Resulting MAE area per atom: 0.7537559694120226
Resulting MAE area per atom sqrt: 0.05012862379833594


# Data evaluated on validation structures

## No Cutoff

### Model lr_1e-3_bs128

In [438]:
relaxed_structures_path = "relaxations/save_model_validation/save_model/no_cutoff_lr_1e-3_bs_128/00496-0.118736-0.038684-0.073790-0.062614relax_struc_0.1_1000_val.pickle"

with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)
print(len(relaxed_structures))

4174


In [436]:
get_error(relaxed_structures, train_structures = reformated_run)

Mean area per atom sqrt: nan
Runs compared: 0
Runs compared volume: 0
Converged Percentage: 0.0%


/home/theorystud01/anaconda3/envs/m3gnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/theorystud01/anaconda3/envs/m3gnet/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: division by zero

# Relaxation with final training structure

## M3GNet

In [345]:
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm import tqdm
import pickle
import numpy as np
import sys
import tensorflow as tf
import sys
import json
from m3gnet.models import M3GNet, Potential, M3GNetCalculator
from m3gnet.trainers import PotentialTrainer
from pymatgen.core import Structure, Lattice
from relax_scripts.m3gnet_dynamics import Relaxer

In [385]:
model_path = "relaxations/save_model/0.01_lr_1e-3_bs_128/00138-0.147841-0.043230-0.096927-0.076845"

In [427]:
def relax_structure(starting_structure, relaxer, max_steps = 100, f_max = 0.1):
    print("Start relaxation")
    print("Structure sites: {}".format(starting_structure.sites))
    adapted_structure = AseAtomsAdaptor.get_atoms(starting_structure)
    trajectory = relaxer.relax(adapted_structure, fmax=f_max, steps=max_steps)
    relaxed_energy = float(trajectory['trajectory'].energies[-1])
    final_structure = trajectory['final_structure']
    if len(trajectory['trajectory'].energies) < max_steps - 1:
        return final_structure, relaxed_energy
    else:
        return None, None

def m3gnet_final_relax(model_path, runs_relaxed, train_run):  
    
    m3gnet = M3GNet.from_dir(model_path)
    potential = Potential(model=m3gnet)
    relaxer = Relaxer(potential=potential, relax_cell=True, optimizer='FIRE', symmetry_constraints = True)

    #######################################
    f_max = 0.1
    maxsteps = 500
    #######################################
    runs_compared = 0
    runs_compared_volume = 0
    
    runs_relaxed = list(relaxed_structures.keys()) #this is list of differnet runs
    
    diff_energy_per_atom = []
    diff_area_per_atom = []
    diff_area_per_atom_sqrt = []
    area_per_atom_sqrt_arr = []
    
    for key in runs_relaxed:
        for id, dictionary in enumerate(train_run):
            if key in dictionary:
                # print(key)
                final_structure_training = Structure.from_dict(train_run[id][key][-1]["structure"])
                nr_sites = len(final_structure_training) #These should be the same to the # of sites in the relaxed structure training set
                final_structure_training_energy = train_run[id][key][-1]["energy"]/nr_sites
                final_structure_relax, final_energy = relax_structure(final_structure_training, relaxer, max_steps = maxsteps, f_max= f_max)
                
                
                struct_train = final_structure_training
                struct_relax = final_structure_relax
                
                ### Volume
                
                if struct_relax != None:
            
                    lat_parm_train = struct_train.lattice.abc
                    lat_parm_relax = struct_relax.lattice.abc

                    area_per_atom_train_sqrt = area_per_atom_sqrt(lat_parm_train, nr_sites) 
                    area_per_atom_relax_sqrt = area_per_atom_sqrt(lat_parm_relax, nr_sites)
                    
                    area_per_atom_train = area_per_atom(lat_parm_train, nr_sites) 
                    area_per_atom_relax = area_per_atom(lat_parm_relax, nr_sites)
                    
                    area_per_atom_sqrt_arr.append(area_per_atom_relax_sqrt)
                    
                    # print(area_per_atom_train)
                    # print(area_per_atom_relax)
                    
                    diff_area_sqrt = np.abs(area_per_atom_train_sqrt - area_per_atom_relax_sqrt)
                    diff_area_org = np.abs(area_per_atom_train - area_per_atom_relax)
                    
                    diff_area_per_atom_sqrt.append(diff_area_sqrt)
                    diff_area_per_atom.append(diff_area_org)
                    
                    runs_compared_volume += 1
                    
                ### Energy
                
                if final_energy != None:
                    final_energy = final_energy/nr_sites
                    diff_energy_per_atom.append(np.abs(final_structure_training_energy - final_energy))
                    # print(final_structure_training_energy, final_structure_relaxation_energy)
                    runs_compared += 1
                    
    print("Runs compared: {}".format(runs_compared))
    print("Runs compared volume: {}".format(runs_compared_volume))
    print("Converged Percentage: {}%".format(np.round(runs_compared/len(runs_relaxed),3)*100))
    # print("Length diff_energy: {}".format(len(diff_energy_per_atom)))
    print("Resulting MAE energy: {}".format(mae(diff_energy_per_atom)))
    print("Resulting MAE area per atom: {}".format(mae(diff_area_per_atom)))
    print("Resulting MAE area per atom sqrt: {}".format(mae(diff_area_per_atom_sqrt)))
    print("Mean area per atom sqrt: {}".format(np.mean(area_per_atom_sqrt_arr)))
    

    return           

In [430]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('INFO')

m3gnet_final_relax(model_path, runs_relaxed[:2], reformated_run)

Start relaxation
Structure sites: [PeriodicSite: O (3.2926, 0.0000, 8.1453) [0.6361, 0.0000, 0.4456], PeriodicSite: O (-1.6463, 2.8515, 8.1453) [0.0000, 0.6361, 0.4456], PeriodicSite: O (0.9418, 1.6313, 8.1453) [0.3639, 0.3639, 0.4456], PeriodicSite: O (-0.0000, 2.9885, 10.7729) [0.3333, 0.6667, 0.5893], PeriodicSite: O (2.5881, 1.4943, 10.7729) [0.6667, 0.3333, 0.5893], PeriodicSite: Ta (1.7741, 0.0000, 9.5973) [0.3427, 0.0000, 0.5250], PeriodicSite: Ta (-0.8871, 1.5364, 9.5973) [0.0000, 0.3427, 0.5250], PeriodicSite: Ta (1.7011, 2.9463, 9.5973) [0.6573, 0.6573, 0.5250], PeriodicSite: Ta (0.0000, 0.0000, 7.4918) [0.0000, 0.0000, 0.4098]]


2023-06-20 15:38:42.129817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_5' with dtype int32 and shape [1]
	 [[{{node graph_5}}]]
2023-06-20 15:38:42.129900: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_6' with dtype int32 and shape [1]
	 [[{{node graph_6}}]]
2023-06-20 15:38:42.129956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_10' with dtype int32 and shape [234]
	 [[{{node graph_10}}]]


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:38:44      -85.200691*       0.7045


2023-06-20 15:38:45.050903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_3' with dtype int32 and shape [?,2]
	 [[{{node graph_3}}]]
2023-06-20 15:38:45.050987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_5' with dtype int32 and shape [1]
	 [[{{node graph_5}}]]
2023-06-20 15:38:45.051031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_6' with dtype int32 and shape [1]
	 [[{{node graph_6}}]]
2023-06-20 15:38:45.

FIRE:    1 15:38:47      -85.210609*       1.8327
FIRE:    2 15:38:47      -85.219719*       0.4719
FIRE:    3 15:38:47      -85.222527*       1.4889
FIRE:    4 15:38:47      -85.224854*       1.1578
FIRE:    5 15:38:47      -85.228165*       0.5666
FIRE:    6 15:38:47      -85.230980*       0.4281
FIRE:    7 15:38:47      -85.233109*       0.7435
FIRE:    8 15:38:47      -85.235786*       1.0520
FIRE:    9 15:38:47      -85.239960*       1.0153
FIRE:   10 15:38:47      -85.245331*       0.6711
FIRE:   11 15:38:48      -85.251320*       0.3812
FIRE:   12 15:38:48      -85.257118*       0.5558
FIRE:   13 15:38:48      -85.263954*       0.7922
FIRE:   14 15:38:48      -85.273140*       0.4351
FIRE:   15 15:38:48      -85.282768*       0.3925
FIRE:   16 15:38:48      -85.292603*       0.8029
FIRE:   17 15:38:48      -85.304993*       0.3122
FIRE:   18 15:38:48      -85.316895*       0.6784
FIRE:   19 15:38:48      -85.331444*       0.3868
FIRE:   20 15:38:48      -85.345451*       1.0087


2023-06-20 15:38:51.564877: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_3' with dtype int32 and shape [?,2]
	 [[{{node graph_3}}]]
2023-06-20 15:38:51.564959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_5' with dtype int32 and shape [1]
	 [[{{node graph_5}}]]
2023-06-20 15:38:51.565004: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_6' with dtype int32 and shape [1]
	 [[{{node graph_6}}]]
2023-06-20 15:38:51.

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:38:54      -55.231182*       0.6863
FIRE:    1 15:38:54      -55.233673*       0.2577
FIRE:    2 15:38:54      -55.233917*       0.1103
FIRE:    3 15:38:54      -55.234203*       0.1066
FIRE:    4 15:38:54      -55.234558*       0.2045
FIRE:    5 15:38:54      -55.235123*       0.1529
FIRE:    6 15:38:54      -55.235775*       0.0998
Start relaxation
Structure sites: [PeriodicSite: Ag (7.8407, 5.0925, 10.4343) [0.6252, 0.6117, 0.5818], PeriodicSite: Ag (7.8405, 5.0935, 7.5007) [0.6249, 0.6119, 0.4182], PeriodicSite: Ag (5.0579, 4.3674, 7.4995) [0.1252, 0.6117, 0.4182], PeriodicSite: Ag (5.0577, 4.3684, 10.4331) [0.1249, 0.6119, 0.5818], PeriodicSite: P (2.5810, 1.2297, 10.0454) [0.3206, 0.1118, 0.5601], PeriodicSite: P (3.1867, 1.3878, 7.8853) [0.4293, 0.1118, 0.4397], PeriodicSite: P (5.3638, 1.9548, 7.8883) [0.8206, 0.1118, 0.4399], PeriodicSite: P (5.9695, 2.1129, 10.0484)

KeyboardInterrupt: 

### Test

In [386]:
def relax_structure(structure_to_relax, model_path):

    m3gnet = M3GNet.from_dir(model_path)
    potential = Potential(model=m3gnet)
    relaxer = Relaxer(potential=potential, relax_cell=True, optimizer='FIRE', symmetry_constraints = True)

    #######################################
    f_max = 0.1
    maxsteps = 500
    #######################################

    def relax_structure(starting_structure, relaxer, max_steps = maxsteps):
        print("Start relaxation")
        print("Structure sites: {}".format(starting_structure.sites))
        adapted_structure = AseAtomsAdaptor.get_atoms(starting_structure)
        trajectory = relaxer.relax(adapted_structure, fmax=f_max, steps=max_steps)
        relaxed_energy = float(trajectory['trajectory'].energies[-1])
        final_structure = trajectory['final_structure']
        if len(trajectory['trajectory'].energies) < max_steps - 1:
            return final_structure, relaxed_energy
        else:
            return None, None

    print("+++++++++++++++++")
    print("Input structure")
    print(structure_to_relax)
    
    # dict_new = {}
    
    final_structure, relaxed_energy = relax_structure(structure_to_relax, relaxer, max_steps = maxsteps)
    
    
    print("Final structure")
    print(final_structure)
    
    return structure_to_relax, final_structure

#     for key, val in tqdm(structure.items()):
#         try:
#             final_structure,energy = relax_structure(val, relaxer)
#             print("Energy: {}\n".format(energy))
#             print("Final structure: {}".format(final_structure))

#             structure_obj = structure[key]
#             dict_new[key] = {"structure": structure_obj, "final_structure": final_structure, "energy": energy}
#             print(dict_new[key])
#         except:
#             print("Relaxation error")
#             problematic_structures.append(structure[key])



In [391]:
idx = 100

key = list(reformated_run[idx])[0]
final_test_structure = Structure.from_dict(reformated_run[idx][key][-1]['structure'])

In [392]:
print(os.getcwd())
relax_structure(final_test_structure, model_path)

/local/scratch/fjager
+++++++++++++++++
Input structure
Full Formula (H6 Pb3)
Reduced Formula: H2Pb
abc   :   5.247479   5.247479  20.957015
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (9)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Pb    0.333333  0.666667  0.698768
  1  Pb    0.666667  0.333333  0.698768
  2  Pb    0         0         0.615567
  3  H     0.372456  0.289888  0.414483
  4  H     0.710112  0.082568  0.414483
  5  H     0.917431  0.627544  0.414483
  6  H     0.289887  0.372456  0.414483
  7  H     0.082569  0.710113  0.414483
  8  H     0.627544  0.917432  0.414483
Start relaxation
Structure sites: [PeriodicSite: Pb (-0.0000, 3.0296, 14.6441) [0.3333, 0.6667, 0.6988], PeriodicSite: Pb (2.6237, 1.5148, 14.6441) [0.6667, 0.3333, 0.6988], PeriodicSite: Pb (0.0000, 0.0000, 12.9005) [0.0000, 0.0000, 0.6156], PeriodicSite: H (1.1939, 1.3174, 8.6863) [0.3725, 0.2899, 0.4145], PeriodicSite: H (

2023-06-20 15:16:09.227944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_5' with dtype int32 and shape [1]
	 [[{{node graph_5}}]]
2023-06-20 15:16:09.228032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_6' with dtype int32 and shape [1]
	 [[{{node graph_6}}]]
2023-06-20 15:16:09.228086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'graph_10' with dtype int32 and shape [120]
	 [[{{node graph_10}}]]


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:16:11      -29.568342*       2.9009
FIRE:    1 15:16:11      -29.578398*       2.2502
FIRE:    2 15:16:11      -29.596260*       1.0796
FIRE:    3 15:16:11      -29.603197*       0.7121
FIRE:    4 15:16:11      -29.603662*       0.6402
FIRE:    5 15:16:11      -29.604456*       0.5051
FIRE:    6 15:16:11      -29.605370*       0.3290
FIRE:    7 15:16:11      -29.606241*       0.3425
FIRE:    8 15:16:11      -29.607033*       0.3551
FIRE:    9 15:16:11      -29.607840*       0.3644
FIRE:   10 15:16:11      -29.608799*       0.3692
FIRE:   11 15:16:11      -29.610073*       0.3688
FIRE:   12 15:16:11      -29.611643*       0.3624
FIRE:   13 15:16:11      -29.613335*       0.3507
FIRE:   14 15:16:11      -29.615097*       0.3369
FIRE:   15 15:16:11      -29.617233*       0.3262
FIRE:   16 15:16:12      -29.620089*       0.3217
FIRE:   17 15:16:12      -29.623476*       0.3209
FI

/home/theorystud01/anaconda3/envs/m3gnet/lib/python3.9/site-packages/pymatgen/io/ase.py:174: UserWarning: Only FixAtoms is supported by Pymatgen. Other constraints will not be set.
  warnings.warn("Only FixAtoms is supported by Pymatgen. Other constraints will not be set.")


In [ ]:
    #print(dict_new)

adendum = "relaxed_structures_{}_{}.pickle".format(f_max, maxsteps)

pickle.dump(dict_new, open(model_path + adendum, 'wb'))

    #with open(model_path+'relaxed_struc.json', 'w') as outfile:
    #    json.dump(dict_new, outfile)    
def write_list(a_list, path):
        # store list in binary file so 'wb' mode
    with open(path, 'wb') as fp:
        pickle.dump(a_list, fp)
            print('Done writing list into a binary file')

## MACE

# Archive

In [ ]:
from pymatgen.core.structure import Structure

# Get a relaxed run key

run_relaxed = list(relaxed_structure_data)[0][0]
print(run_relaxed)


list_keys = list(test_structures)
runs_keys = []
for value in list_keys:
    runs_keys.append(value[0])

print(runs_keys)


for key_relaxed, value_relaxed in relaxed_structure_data:
    print("Key relaxed: {}".format(key_relaxed))
    for key, value in test_structures:
        print("Key test: {}".format(key)
        if key == key_relaxed:
            index = runs_keys.index(key_relaxed)
            print("Index input structures: {}".format(index))
            print("Key structure: {}".format(key))
            energy_structure = test_energies[index]
            print(energy_structure)
            print(value_relaxed['energy'])
            # print("test")



Select the first and last element of each run

In [38]:
first_batch = run_files[0] # this is a list of all the different runs with the geo_opts for the first batch of data
keys_batch = list(first_batch.keys())[:10] 
print(list(first_batch.keys())[:10])
selected_run = first_batch[keys_batch[0]]
# print(selected_run)
reformated_run = {keys_batch[0]: [first_batch[keys_batch[0]][0], first_batch[keys_batch[0]][-1]]}
print(reformated_run)

['runs_1/elements/Ag/Ag/xxx_02k-00_dkepler', 'runs_1/elements/Ag/Ag/xxx_02k-00_phexAl', 'runs_1/elements/Ag/Ag/xxx_02k-00_psqr2Be', 'runs_1/elements/Ag/Ag/xxx_02k-00_psqrAs', 'runs_1/elements/Ag/Ag/xxx_02k-00_pSsBr', 'runs_1/elements/Ag/Ag/xxx_02k-00_skepler', 'runs_1/elements/Ag/Ag/xxx_02p-00_graphene', 'runs_1/elements/Ag/Ag/xxx_02p-00_square', 'runs_1/elements/Al/Al/xxx_02k-00_dkepler', 'runs_1/elements/Al/Al/xxx_02k-00_phexAl']
{'runs_1/elements/Ag/Ag/xxx_02k-00_dkepler': [{'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[5.0609301, 0.06860008, 0.0], [0.07564158, 5.62897846, 0.0], [0.0, 0.0, 17.47820915]], 'pbc': [True, True, True], 'a': 5.061395010079931, 'b': 5.629486668584345, 'c': 17.47820915, 'alpha': 90.0, 'beta': 90.0, 'gamma': 88.45352472892816, 'volume': 497.8261945324156}, 'sites': [{'species': [{'element': 'Ag', 'occu': 1}], 'abc': [0.69937866, 0.7548835, 0.19585474], 'xyz': [3.5966070923475963, 4.29720039333

### Obtain structure information for the test runs

In [ ]:
run_files = []
wd = os.getcwd()
# os.chdir('geo_opt')

for file in tqdm(sorted(os.listdir("data_prep/geo_opt"))[30:32]):
    if file.endswith('json.bz2'):
        file = json.load(bz2.open("data_prep/geo_opt/{}".format(file), 'r'))
        # print(list(file.keys()))
        for i in list(file.keys()):
            # print(i)
            if i in runs_relaxed:
                print(file[i])
                run_files.append(file[i])
                print("run_files added")

        # if file.keys() in runs_relaxed:
        #     run_files.append(json.load(fbz2.open("data_prep/geo_opt/{}".format(file), 'r')))
        #     print("run_file added")


# run_files_new = [run for run in run_files if run in runs_relaxed]